**Practica 1 de laboratorio**

*Formula del área de la elipse*
$$A(a, b) = \frac{a_0}{2} \frac{b_0}{2} \pi $$

*Formula de la propagación de errores*

$$\sigma_A = \bigg[ \bigg( \frac{dA}{da} \Bigg|_{a_0, b_0} \sigma_a \bigg)^2 + \bigg( \frac{dA}{db} \Bigg|_{a_0, b_0} \sigma_b \bigg)^2 \bigg]^{\frac{1}{2}}
$$
$$ \sigma_A = \bigg[ \bigg( \frac{1}{4} b_0 \pi \bigg)^2\sigma_b + \bigg( \frac{1}{4} a_0 \pi \bigg)^2\sigma_a \bigg]^{\frac{1}{2}}